In [1]:
from navigation.navigator import Navigator

from utils import print_models, model_as_str, assumptions_as_str

from clingo.symbol import parse_term

In [2]:
nav = Navigator()

Add the rule "{ a ; b ; c }."

In [3]:
nav.add_rule("{ a ; b ; c }.")

Compute all models

In [4]:
models = nav.compute_models(0)
print_models(models)

Model 1: {}
Model 2: {b}
Model 3: {c}
Model 4: {b, c}
Model 5: {a}
Model 6: {a, c}
Model 7: {a, b}
Model 8: {a, b, c}


Browsing models

In [5]:
i = 0
model = nav.browse_models()
while model is not None:
    i += 1
    print(f"Model {i}: {model_as_str(model)}")
    model = nav.browse_models()
print("No more models\n")

Model 1: {}
Model 2: {b}
Model 3: {c}
Model 4: {b, c}
Model 5: {a}
Model 6: {a, c}
Model 7: {a, b}
Model 8: {a, b, c}
No more models



Consequences and facets

In [6]:
brave = nav.compute_brave_consequences()
print(f"Brave consequences: {model_as_str(brave)}\n")

Brave consequences: {a, b, c}



In [7]:
cautious = nav.compute_cautious_consequences()
print(f"Cautious consequences: {model_as_str(cautious)}\n")


Cautious consequences: {}



In [8]:
facets = nav.compute_facets()
print(f"Facets: {model_as_str(facets)}\n")

Facets: {a, b, c}



Adding assumptions

In [9]:
print("Adding assumption a\n")
nav.add_assumption("a", True)

print("Adding assumption b\n")
nav.add_assumption("b", True)

assumptions = nav.get_assumptions()
print(f"Current assumptions: {assumptions_as_str(assumptions)}\n")

Adding assumption a

Adding assumption b

Current assumptions: {(a, True), (b, True)}



Recomputing consequences/facets and models

In [10]:
facets = nav.compute_facets()
print(f"Facets: {model_as_str(facets)}\n")

cautious = nav.compute_cautious_consequences()
print(f"Cautious consequences: {model_as_str(cautious)}\n")

Facets: {c}

Cautious consequences: {a, b}



In [11]:
models = nav.compute_models(0)
print_models(models)

Model 1: {a, b}
Model 2: {a, b, c}


Extending the logic program

In [12]:
nav.add_rule("d :- a.")

In [13]:
models = nav.compute_models(0)
print_models(models)

Model 1: {a, d, b}
Model 2: {a, d, b, c}


Dropping the current assumptions

In [14]:
assumptions = nav.get_assumptions()
print(f"Current assumptions: {assumptions_as_str(assumptions)}\n")

nav.clear_assumptions()

assumptions = nav.get_assumptions()
print(f"Assumptions after clearing: {assumptions_as_str(assumptions)}\n")

Current assumptions: {(a, True), (b, True)}

Assumptions after clearing: {}



Browsing some models

In [15]:
i = 0
while i < 5:
    model = nav.browse_models()
    i += 1
    print(f"Model {i}: {model_as_str(model)}")
print("Stopped browsing\n")

Model 1: {}
Model 2: {b}
Model 3: {c}
Model 4: {b, c}
Model 5: {a, d}
Stopped browsing



Computing facets

In [16]:
facets = nav.compute_facets()
print(f"Facets: {model_as_str(facets)}\n")

Facets: {a, d, b, c}



Getting all added rules (that can be deactivated)

In [17]:
rules = nav.get_rules()
print(f"Rules: {rules}")

Rules: {'{ a ; b ; c }.': True, 'd :- a.': True}


Deactivate the rule deriving d

In [18]:
nav.deactivate_rule("d :- a.")
rules = nav.get_rules()
print(f"Rules: {rules}")
models = nav.compute_models(0)
print_models(models)

Rules: {'{ a ; b ; c }.': True, 'd :- a.': False}
Model 1: {c}
Model 2: {b, c}
Model 3: {}
Model 4: {b}
Model 5: {a, c}
Model 6: {a, b, c}
Model 7: {a}
Model 8: {a, b}


Activating it again

In [19]:
nav.activate_rule("d :- a.")
print("Browsing models until we find 'd'")
d_symbol = parse_term("d")
found = False
i = 0
model = nav.browse_models()
while model is not None and not found:
    i += 1
    print(f"Model {i}: {model_as_str(model)}")
    if d_symbol in model:
        found = True
        print("Found 'd' in a model")
    model = nav.browse_models()
if not found:
    print("Did not find a model containing 'd'")

Browsing models until we find 'd'
Model 1: {c}
Model 2: {b, c}
Model 3: {}
Model 4: {b}
Model 5: {a, d, c}
Found 'd' in a model


Adding a constraint

In [20]:
nav.add_rule(":- a, b.")

Adding an optimization statement and enabling optimization

In [21]:
nav.add_rule("#maximize{ 1,a : a; 1,b : b; 2,c : c }.", permanent=True)
nav.enable_optimization()
models = nav.compute_models(0)
print_models(models)

Model 1: {b, c}
Model 2: {a, d, c}


Browsing optimal models

In [22]:
i = 0
model = nav.browse_models()
while model is not None:
    i += 1
    print(f"Model {i}: {model_as_str(model)}")
    model = nav.browse_models()
print("No more models\n")

Model 1: {b, c}
Model 2: {a, d, c}
No more models



Disabling optimization in solving

In [23]:
nav.disable_optimization()
models = nav.compute_models(0)
print_models(models)

Model 1: {c}
Model 2: {b, c}
Model 3: {}
Model 4: {b}
Model 5: {a, d, c}
Model 6: {a, d}


Adding multiple rules defining the same atom

In [24]:
nav.add_rule("{ y ; z }.")
nav.add_rule("x :- y.")
nav.add_rule("x :- z.")
nav.add_assumption("x", True)
facets = nav.compute_facets()
print(f"Facets: {model_as_str(facets)}\n")

Facets: {a, d, y, b, c, z}



In [25]:
assumptions = nav.get_assumptions()
print(f"Current assumptions: {assumptions_as_str(assumptions)}\n")

Current assumptions: {(x, True)}



Computing diverse models

In [26]:
models = nav.compute_diverse_models(2)
print_models(models)

Model 1: {z, x}
Model 2: {a, x, d, y, c}


Computing similar models

In [27]:
models = nav.compute_similar_models(2)
print_models(models)

Model 1: {z, x}
Model 2: {b, x, z}
